In [2]:
import pandas
import mido
import os
import re
import shutil

In [3]:
# Use this to strip titles and instrument data of non-alphabetic characters
pattern = "[^a-zA-Z]+"

In [4]:
import os
os.getcwd()
raw_data_folder = os.path.join(os.pardir, "raw_data", "Unzipped")
#raw_data_folder = os.path.join(os.pardir, "test_data")
output_folder = os.path.join(os.pardir, "data")
raw_data_folder

'../raw_data/Unzipped'

In [5]:
def identify_drum_and_guitar(midi):
    guitars = {"guitar", "guita", "gtr", "egtr", "eguitar"}
    drums = {"drums", "drum", "drumsperc", "perc", "percussion"}

    pattern = "[^a-zA-Z ]+"
    has_drum = False
    has_guitar = False

    for track in midi:
        # Check if instruments in track name
        if track.type == "track_name":
            words = {re.sub(pattern, '', word.lower()) for word in track.name.split()}
            if words.intersection(guitars):
                has_guitar = True
            if words.intersection(drums):
                has_drum = True
    if has_drum and has_guitar:
        return True

    # Check if instruments in track text
    for track in midi:
        if track.type == "text":
            words = {re.sub(pattern, '', word.lower())  for word in track.text.split()}
            if words.intersection(guitars):
                has_guitar = True
            if words.intersection(drums):
                has_drum = True
    if has_drum and has_guitar:
        return True

In [6]:
import sys

In [12]:
%time

saved_songs = []
songs = []
errors = 0

for idx, child in enumerate(os.listdir(raw_data_folder)):
    if (idx == 0) or (idx % 250 == 0):
        print(f"Processing file number: {idx}")

    # If the path is a midi file, load the file
    child_path = os.path.join(raw_data_folder, child)
    try:
        if child_path.endswith(".mid"):
            song_path = child_path
            midi_file = mido.MidiFile(song_path, clip=True)
            dg_song = identify_drum_and_guitar(midi_file)
            if dg_song:
                title = re.sub(pattern, "", child[:-4]).lower()
                if title not in saved_songs:
                    saved_songs.append(title)
                    #midi_file.save(f'{output_folder}/{child}')
                    shutil.copyfile(song_path, f'{output_folder}/{child}')


        # If the path is a folder, load the midi files inside
        elif os.path.isdir(child_path):
            for child in os.listdir(child_path):
                song_path = os.path.join(child_path, child)
                if song_path.endswith(".mid"):
                    midi_file = mido.MidiFile(song_path, clip=True)
                    dg_song = identify_drum_and_guitar(midi_file)
                    if dg_song:
                        title = re.sub(pattern, "", child[:-4]).lower()
                        if title not in saved_songs:
                            saved_songs.append(title)
                            #midi_file.save(f'{output_folder}/{child}')
                            shutil.copyfile(song_path, f'{output_folder}/{child}')

    except Exception as ke:
        errors +=1

print(f"Found {len(saved_songs)} .mid files in {raw_data_folder}.")
print(f"Encountered {errors} errors.")

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 9.54 µs


KeyboardInterrupt: 

In [ ]:
%time
for  k, v in drum_and_guitar_songs.items():
    v["midi"].save(f'{output_folder}/{v["filename"]}')

print(f"Saved {len(drum_and_guitar_songs)} midi files to {output_folder}")

zombie
hotelcalifornia
Saved 2 to ../data
